In [1]:
import re

# Parent classes

In [121]:
def is_available_move(c, color: str, coords: tuple, pawn_attack_available=False, pawn=False) -> tuple:   
    """
    Check if the given move is available.

    с: instance of class Chessboard
    coords: tuple of coordinates in format (y, x), where y - number of row, x - number of column
    only_attack: used to control, that pawn can ethier only attack or only move

    returns tuple of bool, 
    where the first bool - is move available, second - if there's enemy
    """
    y = coords[0]
    x = coords[1]
    if all([coord in range(8) for coord in coords]): # checking for going beyond the boundaries of the board
        chessman_on_cell = c.field[y][x]

        # Pawn processing

        if pawn and pawn_attack_available and chessman_on_cell != "." and chessman_on_cell.color != color: # Processing the first move of pawn
            return True, True

        elif pawn and not pawn_attack_available and chessman_on_cell == ".": # Processing the second move of pawn, if the first wasn't attack
            return True, False

        elif pawn and not pawn_attack_available: # If towards is enemy - unavailable move
            return False, False

        # Other chessmen processing
        elif chessman_on_cell == "." : # If the cell is empty - available move
            return True, False
        
        elif chessman_on_cell.color != color: # If on the cell is enemy - available move
            return True, True
    
    return False, False

In [131]:
class Chessman:
    """ 
    Class describes parent class for each chess pieces.

    x: horizontal coord. on the chessboard;
    y: vertical coord. on the chessboard;
    color: color of the chessman ("black" or "white");
    symb: symbol, which is used to visualize chessman on the chessboard;
    
    make_move: describes which moves chessman can make on the chessboard; description of how the figure walks
    fill_moves_map: fills the moves_map list with available moves

    """
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = None
        self.selected = False
        self.moves_map = []

    def __str__(self):
        return self.symb if self.color == "black" else self.symb.upper()

    def set_color(self, color: str) -> None:
        self.color = color

    def set_y(self, y: str) -> None:
        self.y = y

    def make_move(self) -> None:
        pass
    
    def fill_moves_map(self, c, coords: tuple, on_own_part=None, rec_depth=0) -> None:
        """ 
        Fill moves map of the current chessman with coords of available moves.

        c: instance of class Chessboard 
        on_own_part: used only by Pawns
        """
        y = coords[0]
        x = coords[1]

        curr_coords = coords # coords for tracking the possible move of the chessman


        # Pawn
        if self.__class__.__name__ == "Pawn":
            if on_own_part is None:
                on_own_part = self.y in range(4) if self.color == 'white' else self.y in range(5, 8) # check if the pawn on his half of board, only if the variable was not passed

            if rec_depth == 0:
                for attack_coords in self.attack(): # Attack processing, but only if the first move wasn't moving (rec_depth == 0)
                    if all(is_available_move(c=c, color=self.color, coords=attack_coords, pawn_attack_available=True, pawn=True)):
                        self.moves_map.append(attack_coords)

            if any(is_available_move(c=c, color=self.color, coords=self.move_coords(curr_coords), pawn=True)):
                curr_coords = self.move_coords(curr_coords)   
                self.moves_map.append(curr_coords)
                if on_own_part:
                    self.fill_moves_map(c=c, coords=curr_coords, on_own_part=False, rec_depth=1)
        

        # Rook
        initial_coords = coords
        if self.__class__.__name__ == "Rook":
            for changable_coord in ["x", "y"]:
                for direction in [1, -1]:
                    curr_coords = initial_coords
                    is_available = is_available_move(c=c, color=self.color, coords=self.move_coords(curr_coords, changable_coord, direction))
                    while any(is_available):
                        curr_coords = self.move_coords(curr_coords, changable_coord, direction)   
                        self.moves_map.append(curr_coords)

                        if all(is_available): # If on the cell stands enemy, break the cycle cause we cannot attack through enemy
                            break
                        is_available = is_available_move(c=c, color=self.color, coords=self.move_coords(curr_coords, changable_coord, direction))


    def get_moves_map(self, c):
        """
        Show available moves in format Letter Number.

        c: Chessboard
        """
        if len(self.moves_map) < 1:
            return "No available moves"
        else:
            return [c.convert_idx2letters(move_coord) for move_coord in self.moves_map]

## Chessman's child classes

In [132]:
class Pawn(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'p'
        self.direction = 1 if self.color == "white" else -1 # white chessmen on the board go down, black - up
        self.moves_map = [] # list contains all coords in tuple format (y, x), with cells where the chessman can move


    def move_coords(self, coords: tuple) -> tuple:
        """
        One cell forward.

        coords: coords in tuple format (y, x)
        """
        y = coords[0] + self.direction
        x = coords[1]
        return (y, x)


    def attack(self):
        """
        Attack possibility description.
        """
        return [(self.y + self.direction, self.x + self.direction), (self.y + self.direction, self.x - self.direction)]

In [133]:
class Rook(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'r'
        self.moves_map = []
    

    def move_coords(self, coords: tuple, changable_coord: str, direction: int) -> tuple:
        """
        Four perpendicular directions.

        coords: coords in tuple format (y, x)
        changable_coord: y or x, coord, which we change for movement
        direction: 1 or -1, in which direction should chessman move (up or down, right or left)
        """
        if changable_coord == 'y':
            y = coords[0] + direction
            x = coords[1]
        else:
            y = coords[0] 
            x = coords[1] + direction

        return (y, x)

In [134]:
class Knight(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'n'

In [135]:
class Bishop(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'b'

In [136]:
class King(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'k'

        self.in_danger = False # If any enemy chessman can attack the king on the next turn

In [137]:
class Queen(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'q'

# Chessboard class

In [138]:
class Chessboard:
    """
    Class describes properties of the chessboard and its connection with chess pieces.

    field: first dimension is rows, second - columns. Contains blank squares (.) and pieces, each chessman in this list is an instance of the corresponding class;

    """

    def __init__(self) -> None:
        self.field = [["." for _ in range(8)] for _ in range(8)]
        
        self.letters2idx = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7} # dict for converting letters on the board to list's columns indexes
        self.nums2idx = dict(zip([str(8-i) for i in range(8)], [i for i in range(8)])) # dict for converting nums on the board to list's rows indexes

        self.idx2letters = {v:k for k, v in self.letters2idx.items()} # dict for converting list's columns indexes to letters on the board   
        self.idx2nums = {v:k for k, v in self.nums2idx.items()} # dict for converting list's rows indexes to nums on the board  

        self.current_color = 'white' # color whose turn it is to make a move

        self.b_king = None # Black King class instance
        self.w_king = None # White King class instance

        def fill_field() -> None:
            """
            Fill the chessboard with the pieces of both colors.
            """

            for color in ["black", "white"]:
                sp_line_idx, pawns_line_idx = (0, 1) if color == "white" else (7, 6) # Define the indexes of pawns' line and special line according to the color

                king = King(x=4) # Create king instance for variable b_-/w_king

                if color == "black":
                    self.b_king = king
                else:
                    self.w_king = king

                pawns = [Pawn(x=x, y=pawns_line_idx, color=color) for x in range(8)]

                sec_line = [Rook(x=0), Knight(x=1), Bishop(x=2), Queen(x=3), king, Bishop(x=5), Knight(x=6), Rook(x=7)]

                for f in sec_line: # set color and y-coord for all 'special' chessmen
                    f.set_color(color)
                    f.set_y(sp_line_idx)


                self.field[sp_line_idx], self.field[pawns_line_idx] = sec_line, pawns
        
        fill_field()


    def convert_letters2idx(self, letters: str):
        """
        Converts coords from format XN to format (y_field, x_field) (where X - letter, N - number)
        """
        x_field = self.letters2idx[letters[0]]
        y_field = self.nums2idx[letters[1]]

        return (y_field, x_field)

    
    def convert_idx2letters(self, coords: tuple):
        """
        Converts coords from format (y_field, x_field) to format XN (where X - letter, N - number)
        """
        letter = self.idx2letters[coords[1]]
        num = self.idx2nums[coords[0]]

        return letter+num


    def show_desk(self) -> None:
        """ 
        Show the current state of the chessboard. 
        """

        print("   A B C D E F G H")
        print()
        for i in range(len(self.field)):
            print(f"{8-i}", end="  ")
            for j in range(len(self.field[i])):
                print(self.field[i][j], end=" ")
            print("", end=" ")
            print(f"{8-i}")
        print()
        print("   A B C D E F G H",)

    
    def validate_the_input(self, cell: str) -> tuple:
        """
        Return True if the input is valid, False otherwise. 
        """
        prog = re.compile("[1-8A-Ha-h]")
        cell = prog.findall(cell)

        return len(cell) == 2 and (cell[0].isdigit() and cell[1].isalpha() or # check if the entered data is valid
                                    cell[1].isdigit() and cell[0].isalpha())


    def select_the_chessman(self) -> Chessman:
        """ 
        Describes chessman selection mechanism. 
        """

        try:
            cell = input("Enter the coordinates of the cell separated by a space in format 'Letter Number': ")
            if cell == "exit": return # exit from the game
 
            assert self.validate_the_input(cell) == True # check if the entered data is valid

            b_n, b_l = (cell[0], cell[1]) if cell[0].isdigit() else (cell[1], cell[0]) # number, letter of the cell

            row_idx, col_idx = self.nums2idx[b_n], self.letters2idx[b_l.upper()] # converting number and letter to corresponding indexes

            selected_ch = self.field[row_idx][col_idx] # get the chessman from field

            assert selected_ch != '.' and selected_ch.color == self.current_color # check if empty cell or chessman of opponent was selected

        except AssertionError:
            print("Incorrect data entered.")
            return self.select_the_chessman()

        selected_ch.moves_map = []
        selected_ch.fill_moves_map(self, (row_idx, col_idx))
        print(selected_ch.moves_map)

        return selected_ch

    
    def move_the_chessman(self, c: Chessman, coords: tuple) -> None:
        """
        Moves selected chessman on the given cell.

        coords: (y, x)
        """
        self.field[coords[0]][coords[1]] = c
        self.field[c.y][c.x] = '.'
        c.y = coords[0]
        c.x = coords[1]


    def get_chessmen_of_color(self, color: str) -> list:
        """
        Returns list of all chessmen of the given color.
        """
        return [ch for row in self.field for ch in row if ch != '.' and ch.color == color]


    def check_for_a_checkmate(self, king:King) -> bool:
        """
        Checks if the given king in danger (can be attacked on the next turn).
        If yes, checks if there is opportunity for curr_king to avoid death.
        """

        king_coords = (king.y, king.x) # get the coords of king
        king.fill_moves_map(self, king_coords) # fill the moves_map of the current_king
        
        enemy_color = "black" if self.current_color == "white" else "white"
        
        enemy_chessmen = self.get_chessmen_of_color(enemy_color) # get full list of enemies for current_king
        
        enemies_moves_map = []

        for enemy in enemy_chessmen:
            enemy.fill_moves_map(self, (enemy.y, enemy.x))
            enemies_moves_map = enemies_moves_map + enemy.moves_map # fill full enemy moves map


        # if king's coords is in enemies_moves_map, king can be attacked on the next turn ('check' in chess)
        # if all available moves of the king cannot save him - checkmate for curr_king

        return king_coords in enemies_moves_map, set(king_coords).issubset(enemies_moves_map) 


    def turn(self) -> None:
        """
        The course of each turn.
        """

        # Step 0: Checkmate check

        # curr_king = self.b_king if self.current_color == "black" else self.w_king
        
        # is_checkmated = self.check_for_a_checkmate(curr_king)
        
        # if all(is_checkmated):
        #     winner_color = "white" if self.current_color == "black" else "black"
        #     return print(f"{winner_color} won.")
        
        # elif any(is_checkmated):
        #     print(f"Check for the {self.current_color} king.")
        

        # Step 1: Show the current state of the desk.

        self.show_desk()


        # Step 2: Choose the chessman.
        
        selected_ch = self.select_the_chessman()

        chosen_cell = ""

        while not(self.validate_the_input(chosen_cell) and chosen_cell in selected_ch.get_moves_map(self)): # check if the entered data is valid
            chosen_cell = input(f"Choose cell from the following: {selected_ch.get_moves_map(self)}. ") 


        # Step 3: Move the selected chessman on the chosen cell.
        
        coords = self.convert_letters2idx(chosen_cell)
        self.move_the_chessman(selected_ch, coords)

        # Step 4: Pass the move next player

        self.current_color = "white" if self.current_color == "black" else "black"

In [139]:
d = Chessboard()
while True:
    d.turn()

   A B C D E F G H

8  R N B Q K B N R  8
7  P P P P P P P P  7
6  . . . . . . . .  6
5  . . . . . . . .  5
4  . . . . . . . .  4
3  . . . . . . . .  3
2  p p p p p p p p  2
1  r n b q k b n r  1

   A B C D E F G H
Enter the coordinates of the cell separated by a space in format 'Letter Number': A7
[(2, 0), (3, 0)]
Choose cell from the following: ['A6', 'A5']. A5
   A B C D E F G H

8  R N B Q K B N R  8
7  . P P P P P P P  7
6  . . . . . . . .  6
5  P . . . . . . .  5
4  . . . . . . . .  4
3  . . . . . . . .  3
2  p p p p p p p p  2
1  r n b q k b n r  1

   A B C D E F G H
Enter the coordinates of the cell separated by a space in format 'Letter Number': A2
[(5, 0), (4, 0)]
Choose cell from the following: ['A3', 'A4']. A4
   A B C D E F G H

8  R N B Q K B N R  8
7  . P P P P P P P  7
6  . . . . . . . .  6
5  P . . . . . . .  5
4  p . . . . . . .  4
3  . . . . . . . .  3
2  . p p p p p p p  2
1  r n b q k b n r  1

   A B C D E F G H
Enter the coordinates of the cell separated by a s

KeyboardInterrupt: ignored

In [ ]:
d.field[7][0]

In [ ]:
print(d.convert_idx2letters((7, 0)))

In [ ]:
C